# 5. Use the Machine Learning Model

start Spark and instantiate MLFlow

In [ ]:
import os
os.environ['PYSPARK_SUBMIT_ARGS'] = '--packages "com.memsql:memsql-spark-connector_2.11:3.0.0-spark-2.4.4" pyspark-shell'

In [ ]:
import mlflow.spark
import pyspark

spark = pyspark.sql.SparkSession.builder.master("local[*]").config("spark.jars.packages", "org.mlflow.mlflow-spark").getOrCreate()

load the model

In [ ]:
model = mlflow.spark.load_model("spark-model")

grab some data to predict

In [ ]:
spark.conf.set("spark.datasource.memsql.ddlEndpoint", "memsql")
spark.conf.set("spark.datasource.memsql.user", "root")
spark.conf.set("spark.datasource.memsql.password", "")

In [ ]:
data_read = spark.read.format("memsql") \
      .load("tpch.lineitem") \
      .select("*") \
      .filter("prediction is null")

In [ ]:
data_features_only = data_read.drop("prediction")

assemble features - input parameters to the model

In [ ]:
from pyspark.ml.feature import VectorAssembler

feature_columns = ['l_partkey','l_suppkey','l_quantity','l_discount','l_tax']
assembler = VectorAssembler(inputCols=feature_columns,outputCol="features")
data_to_predict = assembler.transform(data_features_only)

In [ ]:
from pyspark.ml.regression import LinearRegression
algo = LinearRegression(featuresCol="features", labelCol="prediction")

run the predictions

In [ ]:
predictions = model.transform(data_to_predict)

remove the features column

In [ ]:
results = predictions.drop("features")

write the finished rows into place in the db

In [ ]:
results.write.format("memsql") \
    .option("overwriteBehavior", "merge") \
    .option("loadDataCompression", "LZ4") \
    .mode("overwrite") \ 
    .save("tpch.lineitem")